In [1]:
## Import required Python modules
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy, scipy.stats
import io
import base64
#from IPython.core.display import display
from IPython.display import display, HTML, Image
from urllib.request import urlopen

try:
    import astropy as apy
    import astropy.table
    _apy = True
    #print('Loaded astropy')
except:
    _apy = False
    #print('Could not load astropy')

## Customising the font size of figures
plt.rcParams.update({'font.size': 14})

## Customising the look of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))
## This custom file is adapted from https://github.com/lmarti/jupyter_custom/blob/master/custom.include
HTML('custom.css')
#HTML(urlopen('https://raw.githubusercontent.com/bretonr/intro_data_science/master/custom.css').read().decode('utf-8'))

In [2]:
HTML('''
<script>
    function toggleCodeCells() {
      var codeCells = document.querySelectorAll('.jp-CodeCell');

      codeCells.forEach(function(cell) {
        var inputArea = cell.querySelector('.jp-InputArea');
        if (inputArea) {
          var currentDisplay = inputArea.style.display || getComputedStyle(inputArea).display;
          inputArea.style.display = currentDisplay === 'none' ? '' : 'none';
        }
      });
    }
</script>

<!-- Add a button to toggle visibility of input code cells -->
<button onclick="toggleCodeCells()">Toggle Code Cells</button>
''')

<div class="container-fluid">
    <div class="row">
        <div class="col-md-8" align="center">
            <h1>PHYS 10791: Introduction to Data Science</h1>
            <!--<h3>2019-2020 Academic Year</h3><br>-->
        </div>
        <div class="col-md-3">
            <img align='center' style="border-width:0" src="images/UoM_logo.png"/>
        </div>
    </div>
</div>

<div class="container-fluid">
    <div class="row">
        <div class="col-md-2" align="right">
            <b>Course instructors:&nbsp;&nbsp;</b>
        </div>
        <div class="col-md-9" align="left">
            <a href="http://www.renebreton.org">Prof. Rene Breton</a> - Twitter <a href="https://twitter.com/BretonRene">@BretonRene</a><br>
            <a href="http://www.hep.manchester.ac.uk/u/gersabec">Dr. Marco Gersabeck</a> - Twitter <a href="https://twitter.com/MarcoGersabeck">@MarcoGersabeck</a>
        </div>
    </div>
</div>

# Chapter 6 - Summary

## 6.1 Errors

### 6.1.1 Types of errors

Two types of errors:
- **Statistical errors**: repeated measurements would improve how well the value can be known.
- **Systematic errors**: will not improve even if more data are collected.

### 6.1.2 Propagation of errors

Given a quantity $x$ that has a measurement error characterised by a variance $V(x)$. The error propagation to a function $f(x)$ which depends on that variable will yield to a variance (i.e. error) on that function which is $V(f)$:
\begin{equation}
  V(f) \approx \left( \frac{\partial f}{\partial x} \right)^2 V(x) \,.
\end{equation}

If the function includes more than one quantities which have uncertainties, the error propagation to a function would become:
\begin{equation}
  V(f) \approx \left( \frac{\partial f}{\partial x} \right)^2 V(x) + \left( \frac{\partial f}{\partial y} \right)^2 V(y) + \dots + 2 \left( \frac{\partial f}{\partial x} \right) \left( \frac{\partial f}{\partial y} \right) \operatorname{cov} (x,y) + \dots
\end{equation}

## 6.2 Monte Carlo

### 6.2.1 Uniformly distributed random numbers

Generating random uniform integers between $0$ and $m-1$ using the **linear congruential generator (LCG)**:
\begin{equation}
  n_{i} = (a n_{i-1} + c) \operatorname{mod} m \,,
\end{equation}
where $a$, $c$ and $m$ are provided constants, $n_{i-1}$ is the previous random number, and $n_i$ is the current one.

The above can be turned into fractional number between $0$ and $1$ as follows:
\begin{equation}
  r_{i} = \frac{n_i}{m} \,.
\end{equation}

##### Recall
The LCG will repeat after at most $m$ draws. Once a number repeats, then the full sequence keeps repeating.

### 6.2.2 Numerical integration

Consider the integral:
\begin{equation}
  I = \int_a^b f(x) dx \,.
\end{equation}

##### Procedure for trapezoid integration
\begin{equation}
  I \approx \frac{b-a}{n} \sum_{i=1}^{n} f(x_i)
\end{equation}

with

\begin{equation}
  x_i = a + \left( i-\frac{1}{2} \right)\frac{(b-a)}{n} \,.
\end{equation}

##### Procedure for 'crude' Monte Carlo integration
- Generate $n$ random numbers $r_i$ drawn from a uniform distribution $U(0,1)$.
- Evaluate the integral according to:

  \begin{equation}
    I \approx \frac{b-a}{n} \sum_{i=1}^{n} f(x_i)
  \end{equation}
  with $x_i = a + r_i(b-a)$.

##### Procedure for rejection sampling integration

1. Enclose the function to be integrated into a virtual box spanning the integration boundaries $a$ and $b$ in $x$ and $0$ and $f_{\rm max}$ in $y$, where $f_{\rm max}$ is at least as large of the maximum value of the function.
2. Generate a random number $x_i$, uniformally distributed between $a$ and $b$, i.e. $x_i = a + r_i(b-a)$ with $r_i$ drawn from a uniform distribution $U(0,1)$.
3. Generate a second random number $u_i$ drawn from a uniform distribution $U(0,1)$.
4. If $u_i < f(x_i)/f_{\rm max}$, then accept $x_i$ (i.e. the point is below the function).

Repeat steps 2, 3 and 4 $n$ times. The value of the integral can then by computed directly:
\begin{equation}
  I = \int_a^b f(x) {\rm d}x \approx \frac{n_{\rm accepted}}{n} (b-a) f_{\rm max} \,.
\end{equation}

### 6.2.3 Sampling from an arbitrary probability distribution

##### Procedure to generate random samples from inverse transform sample

1. Given a function that you want to sample from, $f(x)$, ensure that the function is normalised.
2. Calculate the cummulative distribution function from the minimum value that this function can take up to a certain value $X$ and set it equal to $r$: $\int_{\rm x_{min}}^X f(x) \,{\rm d}x = r$
3. Solve for $X$.
4. Draw a uniform random number between 0 and 1, $r \sim U(0,1)$, and insert it in the equation for $X$ from above. You now have a random sample that was draw from the original distribution, $X \sim f(x)$.

Repeat step 4 as many times as you need random samples.

<div class="well" align="center">
    <div class="container-fluid">
        <div class="row">
            <div class="col-md-3" align="center">
                <img align="center" alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" width="60%">
            </div>
            <div class="col-md-8">
            This work is licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>).
            </div>
        </div>
    </div>
    <br>
    <br>
    <i>Note: The content of this Jupyter Notebook is provided for educational purposes only.</i>
</div>